In [4]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.font_manager as fm
from sklearn.linear_model import LinearRegression

In [5]:
# 1. 폰트 파일 경로 설정
font_path = r"C:\Users\User\Desktop\MyRepo\Portfolio\Portfolio_ver2\5_fonts\Pretendard-Medium.otf"

# 2. 폰트 이름 가져오기
font_name = fm.FontProperties(fname = font_path).get_name()
fm.fontManager.addfont(font_path)

# 3. Matplotlib의 기본 폰트로 설정
plt.rc("font", family = font_name)

# 4. 마이너스 기호 깨짐 방지
plt.rcParams["axes.unicode_minus"] = False

print(f"설정된 폰트 이름: {font_name}")

설정된 폰트 이름: Pretendard


In [6]:
df = pd.read_csv(r"C:\Users\User\Desktop\MyRepo\Portfolio\Portfolio_ver2\1_data\processed\df_final_v2.csv")
df["datetime"] = pd.to_datetime(df["datetime"])
df["강수량(mm)"] = df["강수량(mm)"].fillna(0)

In [7]:
# ── ① 기온 1°C 상승 시 전력수요 변화 ────────────
lr = LinearRegression()
lr.fit(df[["기온(°C)"]], df["전력사용량(MWh)"])
coef = round(lr.coef_[0], 1)
print(f"① 기온 1°C 상승 시 전력수요: {coef} MWh 변화")

# ── ② 공휴일 vs 평일 수요 차이 ──────────────────
weekday = df[df["is_off"] == 0]["전력사용량(MWh)"].mean()
off_day = df[df["is_off"] == 1]["전력사용량(MWh)"].mean()
diff_pct = round((weekday - off_day) / weekday * 100, 1)
print(f"② 평일 평균: {weekday:.0f} MWh / 주말·공휴일 평균: {off_day:.0f} MWh")
print(f"   → 주말·공휴일이 평일 대비 {diff_pct}% 낮음")

# ── ③ 계절별 피크 시간대 ────────────────────────
def get_season(m):
    if m in [3,4,5]: return "봄"
    elif m in [6,7,8]: return "여름"
    elif m in [9,10,11]: return "가을"
    else: return "겨울"

df["season"] = df["datetime"].dt.month.apply(get_season)

for season in ["봄", "여름", "가을", "겨울"]:
    peak_hour = df[df["season"] == season].groupby("hour")["전력사용량(MWh)"].mean().idxmax()
    peak_val  = df[df["season"] == season].groupby("hour")["전력사용량(MWh)"].mean().max()
    print(f"③ {season} 피크: {peak_hour}시 ({peak_val:.0f} MWh)")

# ── ④ 폭염일 (33°C 이상) vs 일반일 수요 차이 ────
normal  = df[df["기온(°C)"] < 33]["전력사용량(MWh)"].mean()
extreme = df[df["기온(°C)"] >= 33]["전력사용량(MWh)"].mean()
diff    = round(extreme - normal, 0)
diff_p  = round((extreme - normal) / normal * 100, 1)
print(f"④ 일반일 평균: {normal:.0f} MWh / 폭염일 평균: {extreme:.0f} MWh")
print(f"   → 폭염일이 일반일 대비 {diff:.0f} MWh ({diff_p}%) 높음")

# ── ⑤ 미세먼지 등급별 수요 차이 ─────────────────
def dust_grade(v):
    if v <= 30: return "좋음"
    elif v <= 80: return "보통"
    elif v <= 150: return "나쁨"
    else: return "매우나쁨"

df["dust_grade"] = df["미세먼지(PM10)"].apply(dust_grade)
dust = df.groupby("dust_grade")["전력사용량(MWh)"].mean().reindex(["좋음","보통","나쁨","매우나쁨"])
for grade, val in dust.items():
    print(f"⑤ 미세먼지 {grade}: 평균 {val:.0f} MWh")

① 기온 1°C 상승 시 전력수요: 38.3 MWh 변화
② 평일 평균: 5333 MWh / 주말·공휴일 평균: 4563 MWh
   → 주말·공휴일이 평일 대비 14.4% 낮음
③ 봄 피크: 12시 (5126 MWh)
③ 여름 피크: 14시 (7355 MWh)
③ 가을 피크: 18시 (5859 MWh)
③ 겨울 피크: 11시 (6597 MWh)
④ 일반일 평균: 5046 MWh / 폭염일 평균: 8580 MWh
   → 폭염일이 일반일 대비 3533 MWh (70.0%) 높음
⑤ 미세먼지 좋음: 평균 5165 MWh
⑤ 미세먼지 보통: 평균 5049 MWh
⑤ 미세먼지 나쁨: 평균 4545 MWh
⑤ 미세먼지 매우나쁨: 평균 4446 MWh


## 📊 주요 인사이트 및 운영 대책

---

### ① 기온 1°C 상승 → 전력수요 38.3 MWh 증가

**분석 결과**  
기온이 1°C 오를 때마다 서울시 전력수요는 평균 38.3 MWh 증가하는 선형 관계를 확인했습니다.  
기상청 기온 예보를 활용한 익일 수요 사전 예측이 가능합니다.

**운영 대책**  
- 기온 예보 기반 익일 수요 예측 자동화
- ESS 방전 스케줄 사전 편성

---

### ② 주말·공휴일 수요 평일 대비 14.4% 감소

**분석 결과**  
| 구분 | 평균 수요 | 차이 |
|------|-----------|------|
| 평일 | 5,333 MWh | 기준 |
| 주말·공휴일 | 4,563 MWh | -770 MWh (-14.4%) |

산업·상업용 전력 비중이 높아 업무일 여부에 따라 수요 차이가 뚜렷합니다.

**운영 대책**  
- 연휴 기간 발전기 예비력 재편성
- 불필요한 대기 발전 최소화로 연료비 절감

---

### ③ 계절별 피크 시간대 상이

**분석 결과**  
| 계절 | 피크 시간 | 피크 수요 |
|------|-----------|-----------|
| 봄 | 12시 | 5,126 MWh |
| 여름 | 14시 | 7,355 MWh |
| 가을 | 18시 | 5,859 MWh |
| 겨울 | 11시 | 6,597 MWh |

여름은 오후 냉방 피크, 겨울은 오전 난방 피크, 가을은 퇴근 후 저녁 피크로 계절마다 패턴이 다릅니다.

**운영 대책**  
- 계절별 피크 시간에 맞춘 DR 발령 기준 시간대 차별화
- ESS 방전 타이밍 계절별 최적화

---

### ④ 폭염일(33°C 이상) 수요 일반일 대비 70% 급증

**분석 결과**  
| 구분 | 평균 수요 | 차이 |
|------|-----------|------|
| 일반일 | 5,046 MWh | 기준 |
| 폭염일 (33°C↑) | 8,580 MWh | +3,533 MWh (+70.0%) |

폭염 시 냉방 수요가 폭발적으로 증가하며, 이는 계통 안정성에 가장 큰 위협 요인입니다.

**운영 대책**  
- 폭염 예보 D-1 기준 비상 수요대응(DR) 사전 발령
- 대형 건물 냉방 온도 제한 협조 요청
- ESS 방전 스케줄 14시 집중 편성